In [1]:
import sys
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns 

windows_dir = "../kaggle/all/"
ubuntu_dir = "/home/prof/kaggle/unbankedcredit/kaggle/all/"

import platform

system = platform.system()

print("Notebook is running on operating system: {}".format(system))
if system == "Linux":
    directory = ubuntu_dir
else:
    directory = windows_dir

Notebook is running on operating system: Linux


In [2]:
!pwd

/home/prof/kaggle/unbankedcredit


In [3]:
# Training data
train_file = "/home/prof/kaggle/unbankedcredit/train_enhanced.csv"
train_pd = pd.read_csv(train_file)
print('Training data shape: ', train_pd.shape)
train_pd.head()

Training data shape:  (307511, 245)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET,DAYS_EMPLOYED_ANOM,EXT_SOURCE_1_2_3_PRODUCT,EXT_SOURCE_1_3_PRODUCT,EXT_SOURCE_1_3_ADD,INCOME_RATIO,TOTAL_OUTSTANDING_DEBT
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,1,0,1,False,0.003043,0.011573,0.222413,2.007889,245781.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,1,0,0,False,NaN,NaN,NaN,4.790750,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,False,NaN,NaN,NaN,2.000000,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,False,NaN,NaN,NaN,2.316167,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,False,NaN,NaN,NaN,4.222222,0.0


In [4]:
# Test data
test_file = "/home/prof/kaggle/unbankedcredit/test_enhanced.csv"
test_pd = pd.read_csv(test_file)
print('Training data shape: ', test_pd.shape)
test_pd.head()

Training data shape:  (48744, 244)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM,EXT_SOURCE_1_2_3_PRODUCT,EXT_SOURCE_1_3_PRODUCT,EXT_SOURCE_1_3_ADD,INCOME_RATIO,TOTAL_OUTSTANDING_DEBT
0,100001,0,0,1,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,1,0,1,0,False,0.094803,0.120057,0.912134,4.213333,596686.5
1,100005,0,0,1,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,0,0,0,0,False,0.071345,0.244619,0.997952,2.250182,568408.5
2,100013,0,1,1,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,0,0,0,0,False,NaN,NaN,NaN,3.275378,0.0
3,100028,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,0,0,1,0,False,0.164177,0.322119,1.138438,5.000000,186304.5
4,100038,0,1,0,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,0,0,0,0,False,NaN,NaN,NaN,3.475000,NaN


In [5]:
train_data = train_pd[['SK_ID_CURR','AMT_INCOME_TOTAL', 'TARGET', 'EXT_SOURCE_1_2_3_PRODUCT', 'TOTAL_OUTSTANDING_DEBT' ]]

In [6]:
test_data = test_pd[['SK_ID_CURR','AMT_INCOME_TOTAL', 'EXT_SOURCE_1_2_3_PRODUCT', 'TOTAL_OUTSTANDING_DEBT' ]]

In [7]:
# from sklearn import preprocessing 
# X_imputed=preprocessing.Imputer().fit_transform(train_data) 
# X_imputed.shape

In [8]:
# from sklearn import preprocessing 
# Y_imputed=preprocessing.Imputer().fit_transform(test_data) 
# Y_imputed.shape

In [9]:
from sklearn.preprocessing import MinMaxScaler, Imputer

train_labels = train_data['TARGET']

# Drop the target from the training data
if 'TARGET' in train_data:
    train = train_data.drop(columns = ['TARGET'])
else:
    train = train_data.copy()
    
# Feature names
features = list(train_data.columns)

# Copy of the testing data
test = test_data.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train_model = imputer.transform(train)
test_model = imputer.transform(test)


# Repeat with the scaler
scaler.fit(train_model)
s_train = scaler.transform(train_model)
s_test = scaler.transform(test_model)

print('Training data shape: ', s_train.shape)
print('Testing data shape: ', s_test.shape)

Training data shape:  (307511, 4)
Testing data shape:  (48744, 4)


In [10]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(s_train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [11]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(s_test)[:, 1]

In [12]:
print(len(log_reg_pred))
print(len(s_test))

48744
48744


In [13]:
# Submission dataframe
submit = test_data[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
submit

,SK_ID_CURR,TARGET
0,100001,0.169511
1,100005,0.171648
2,100013,0.167250
3,100028,0.163312
4,100038,0.167242
5,100042,0.167201
6,100057,0.153121
7,100065,0.168281
8,100066,0.151390
9,100067,0.176547


In [15]:
X = s_train
y = train_labels

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X, y)  
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
clf.predict(s_test)

In [87]:
# # Save the submission to a csv file
# submit.to_csv('log_reg_baseline2.csv', index = False)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from itertools import cycle

# from sklearn import svm, datasets
# from sklearn.metrics import roc_curve, auc
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import label_binarize
# from sklearn.multiclass import OneVsRestClassifier
# from scipy import interp


# X = s_train
# y = train_labels

# # Binarize the output
# n_classes = y.shape

# # Add noisy features to make the problem harder
# random_state = np.random.RandomState(0)
# n_samples, n_features = X.shape
# X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# # shuffle and split training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
#                                                     random_state=0)

# # Learn to predict each class against the other
# classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
#                                  random_state=random_state))
# y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# # Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# plt.figure()
# lw = 2
# plt.plot(fpr[2], tpr[2], color='darkorange',
#          lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()